# Issue 8

Issue 8 shows some artifacts in the horizon calculation. This seems to step from the `hor1d` function not looking at the adjacent pixel or not searching the full length of the vector.

In [ ]:
import numpy as np
import rasterio as rio
import pandas as pd
import matplotlib.pyplot as plt

from topocalc.horizon import horizon

In [ ]:
ras = rio.open('data/test_proj_windowed_int.tif')
ar = ras.read(1)
ar = ar.astype(np.float)

# plt.figure(figsize=(15,15))
# plt.imshow(ar)
# plt.colorbar()

In [ ]:
horizon_test_azimuth_0 = np.rad2deg(np.arccos(horizon(0, ar, 30)))

plt.figure(figsize=(15,15))
plt.imshow(horizon_test_azimuth_0)
plt.colorbar()

In [ ]:
# Pull out into 2D, look at this particular column that has a 90 degree horizon
idx = 519
surf = ar[:, idx]
hor = horizon(0, ar, 30)[:, idx]

df = pd.DataFrame([surf, hor], index=['surface', 'horizon']).transpose()

fig,ax = plt.subplots(figsize=(20,10))
df.surface.plot(ax=ax)
df.horizon.plot(ax=ax, secondary_y=True)
ax.legend([ax.get_lines()[0], ax.right_ax.get_lines()[0]], df.columns)